In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class LeftoverCoarseLabelFinal(dj.Computed):
    definition = """
    # Ensure that Vertex labels for pinky.LeftoverProofreadLabel match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.LeftoverProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.LeftoverProofreadLabel
    """
    
    key_source = pinky.LeftoverProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.LeftoverProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.ExcitatoryLeftoverMeshes & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [4]:
start = time.time()
LeftoverCoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346349498482
Segment 648518346349498482 vertex labels regenerated in: 0.34923624992370605 seconds.
648518346349498566
Segment 648518346349498566 vertex labels regenerated in: 0.2451915740966797 seconds.
648518346349498835
Segment 648518346349498835 vertex labels regenerated in: 0.4676191806793213 seconds.
648518346349499369
Segment 648518346349499369 vertex labels regenerated in: 0.26751041412353516 seconds.
648518346349499636
Segment 648518346349499636 vertex labels regenerated in: 0.2737879753112793 seconds.
648518346349499680
Segment 648518346349499680 vertex labels regenerated in: 0.2221837043762207 seconds.
648518346349499701
Segment 648518346349499701 vertex labels regenerated in: 0.24158930778503418 seconds.
648518346349499783
Segment 648518346349499783 vertex labels regenerated in: 1.3753173351287842 seconds.
648518346349500120
Segment 648518346349500120 vertex labels regenerated in: 0.6178379058837891 seconds.
648518346349500324
Segment 648518346349500324 vertex labels r

In [5]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()